## Unsupervised Learning Trading

- Load and preprocess data
- Feature extraction per item
- Aggregate and filter for top items
- Calculate returns per month
- Load Fama-French Factors and Rolling Factor Bias
- K-Means Cluster per month ~ Efficient Frontier max sharpe ratio optimization
- Visualize data

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
import sklearn as sk
import pandas_ta
import warnings
warnings.filterwarnings("ignore")
from statsmodels.regression.rolling import RollingOLS
from pypfopt.efficient_frontier import EfficientFrontier

In [12]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [14]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
symb_list = sp500['Symbol'].unique().tolist()

In [21]:
## range of data, end date and start date set to 8yrs prior
end_date = '2025-06-27'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)
start_date, end_date

(Timestamp('2017-06-29 00:00:00'), '2025-06-27')

In [30]:
df = yf.download(tickers=symb_list, 
                 start=start_date, 
                 end=end_date).stack()

[*********************100%***********************]  503 of 503 completed


In [33]:
### multi-indexing by stack above, for organization
### indexing to match the new index
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

### Calculate features and tech indicators
- Garman-Klass Volatility
- RSI
- Bollinger Bands
- ATR
- MACD
- Dollar Volume

In [34]:
df

Price                   close        high         low        open       volume
date       ticker                                                             
2017-06-29 A        55.353146   56.134493   55.061320   55.870904    1356800.0
           AAPL     33.467327   33.805078   33.141227   33.707248  125997600.0
           ABBV     51.036434   51.522291   50.635072   51.508210    4444700.0
           ABT      42.087963   42.493902   41.889313   42.087963    6048500.0
           ACGL     29.848703   29.981830   29.734597   29.975490    1898100.0
...                       ...         ...         ...         ...          ...
2025-06-26 XYZ      65.110001   65.205002   63.800999   64.879997    5097200.0
           YUM     147.050003  149.009995  145.649994  146.899994    3297100.0
           ZBH      91.739998   92.830002   90.610001   92.669998    2822000.0
           ZBRA    308.320007  308.660004  302.820007  303.429993     443200.0
           ZTS     155.270798  157.094701  154.742565  156.955168    3122700.0

[991801 rows x 5 columns]

In [35]:
def garman_klass_volatility(df, window=21):
    log_hl = np.log(df['high'] / df['low'])
    log_co = np.log(df['close'] / df['open'])
    rs = 0.5 * log_hl ** 2 - (2 * np.log(2) - 1) * log_co ** 2
    # Calculate rolling sum for volatility estimate
    gk_vol = rs.groupby('ticker').rolling(window=window, min_periods=1).sum().reset_index(level=0, drop=True)
    gk_vol = np.sqrt(gk_vol)
    df['gk_volatility'] = gk_vol
    return df

df = garman_klass_volatility(df)

In [36]:
df

Price                   close        high         low        open  \
date       ticker                                                   
2017-06-29 A        55.353146   56.134493   55.061320   55.870904   
           AAPL     33.467327   33.805078   33.141227   33.707248   
           ABBV     51.036434   51.522291   50.635072   51.508210   
           ABT      42.087963   42.493902   41.889313   42.087963   
           ACGL     29.848703   29.981830   29.734597   29.975490   
...                       ...         ...         ...         ...   
2025-06-26 XYZ      65.110001   65.205002   63.800999   64.879997   
           YUM     147.050003  149.009995  145.649994  146.899994   
           ZBH      91.739998   92.830002   90.610001   92.669998   
           ZBRA    308.320007  308.660004  302.820007  303.429993   
           ZTS     155.270798  157.094701  154.742565  156.955168   

Price                   volume  gk_volatility  
date       ticker                              
2017-06-29 A         1356800.0       0.012362  
           AAPL    125997600.0       0.013303  
           ABBV      4444700.0       0.010870  
           ABT       6048500.0       0.010133  
           ACGL      1898100.0       0.005229  
...                        ...            ...  
2025-06-26 XYZ       5097200.0       0.081818  
           YUM       3297100.0       0.055125  
           ZBH       2822000.0       0.058594  
           ZBRA       443200.0       0.061766  
           ZTS       3122700.0       0.062432  

[991801 rows x 6 columns]